In [26]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw 
from rdkit.Chem import rdmolops
from rdkit.Chem.Draw import IPythonConsole
import numpy as np

In [47]:
vyb = Chem.SDMolSupplier('sets/bzr_3d.sd')

ms = [x for x in vyb if x is not None]
len(ms)

405

In [48]:
def clean_mark(mark, n) :
    
    for i in range(n) :
        mark[i] = 0

In [49]:
def wave(a, ii, n, mark) :
    
    f = 1
    
    mark_num = 1

    # поджог
    mark[ii] = mark_num

    # пока не подожжем все вершины
    while 1 :
        if f == 0 :
            break

        f = 0

        for i in range (n) :
            if mark[i] == mark_num :
                for j in range (n) :
                    if a[i][j] == 1 and mark[j] == 0 :
                        mark[j] = mark_num + 1
    
        for i in range (n) :
            if mark[i] == 0 :
                f = 1

        mark_num = mark_num + 1

In [50]:
def init_distance_table_i(d, mark, i, n) :

    for j in range(n) :
        d[i][j] = mark[j] - 1

In [51]:
def init_distance_table(a, d, mark, n) :

    for i in range(n) :
        # очистить метки
        clean_mark(mark, n)

        # волна со стартовым атомом i
        wave(a, i, n, mark)

        # заполнить i-тую строку матрицы расстояний
        init_distance_table_i(d, mark, i, n)

In [52]:
# инициализация матрицы размера n x n
def init_matrix(a, n) :
    
    for i in range (n) :
        for j in range (n) :
            a[i, j] = 0

In [53]:
# инициализация массива длины n
def init_array(a, n) :
    for i in range (n) :
        a[i] = 0

In [54]:
def find_diam_and_rad (d, n) :

    loc_max = np.zeros(n, dtype = int, order = 'C')
    init_array(loc_max, n)

    for i in range(n) :
        max = 0

        for j in range(n) :
            if d[i][j] > max:
                max = d[i][j]

        loc_max[i] = max

    rad = loc_max[0]
    diam = loc_max[0]

    for i in range(n) :
        if loc_max[i] < rad :
            rad = loc_max[i]

        if loc_max[i] > diam :
            diam = loc_max[i]
            
    return diam, rad

In [55]:
diam = np.zeros(len(ms), dtype = int, order = 'C')
init_array(diam, len(ms))

rad = np.zeros(len(ms), dtype = int, order = 'C')
init_array(rad, len(ms))

for i in range(len(ms)) :
    n = ms[i].GetNumAtoms()
    a = Chem.rdmolops.GetAdjacencyMatrix(ms[i], useBO=False, emptyVal=0, force=False, prefix='')
     
    d = np.zeros((n, n), dtype = int, order = 'C')
    init_matrix(d, n)
    
    mark = np.zeros(n, dtype = int, order = 'C')
    init_array(mark, n)
    
    init_distance_table(a, d, mark, n)
    
    diam[i], rad[i] = find_diam_and_rad(d, n)
    
    #print (diam[i], rad[i])

In [56]:
np.savetxt('bzr-diam_rad.csv', [p for p in zip(diam, rad)], delimiter=',', fmt='%d')